# Computing Trajectory Metrics

<img align="right" src="https://movingpandas.github.io/movingpandas/assets/img/movingpandas.png">

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/movingpandas/movingpandas/main?filepath=tutorials/6-trajectory-metrics.ipynb)

**<p style="color:#e31883">This notebook demonstrates the current development version of MovingPandas.</p>**

For tutorials using the latest release visit https://github.com/movingpandas/movingpandas-examples.


In [1]:
import urllib
import os
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon, MultiPoint
from datetime import datetime, timedelta
from holoviews import opts, dim

import sys
sys.path.append("..")
import movingpandas as mpd
mpd.show_versions()

import warnings
warnings.simplefilter("ignore")

h:\miniconda3\envs\movingpandas\lib\site-packages\geoviews\operation\__init__.py:14: HoloviewsDeprecationWarning: 'ResamplingOperation' is deprecated and will be removed in version 1.17, use 'ResampleOperation2D' instead.
  from holoviews.operation.datashader import (



MovingPandas 0.17.0

SYSTEM INFO
-----------
python     : 3.10.11 | packaged by conda-forge | (main, May 10 2023, 18:51:25) [MSC v.1934 64 bit (AMD64)]
executable : h:\miniconda3\envs\movingpandas\python.exe
machine    : Windows-10-10.0.19045-SP0

GEOS, GDAL, PROJ INFO
---------------------
GEOS       : None
GEOS lib   : None
GDAL       : 3.6.4
GDAL data dir: None
PROJ       : 9.2.0
PROJ data dir: H:\miniconda3\envs\movingpandas\Library\share\proj

PYTHON DEPENDENCIES
-------------------
geopandas  : 0.13.0
pandas     : 2.0.1
fiona      : 1.9.3
numpy      : 1.23.5
shapely    : 2.0.1
rtree      : 1.0.1
pyproj     : 3.5.0
matplotlib : 3.7.1
mapclassify: 2.5.0
geopy      : 2.3.0
holoviews  : 1.16.0
hvplot     : 0.8.3
geoviews   : 1.9.6
stonesoup  : 0.1b12


## Raw AIS trajectories

In [2]:
gdf = gpd.read_file('data/demodata_ais.gpkg')
gdf['t'] = pd.to_datetime(gdf['Timestamp'], format='%d/%m/%Y %H:%M:%S')
gdf = gdf[gdf.SOG>0]
traj_collection = mpd.TrajectoryCollection(gdf, 'MMSI', min_length=100, t='t')
print("Finished creating {} trajectories".format(len(traj_collection)))

Finished creating 77 trajectories


## Split into trajectories representing individual trips

In [3]:
trips = mpd.ObservationGapSplitter(traj_collection).split(gap=timedelta(minutes=30))
print("Extracted {} individual trips from {} continuous vessel tracks".format(len(trips), len(traj_collection)))

Extracted 218 individual trips from 77 continuous vessel tracks


## Convert to trajectory GeoDataFrame with summary statistics

In [4]:
traj_gdf = trips.to_traj_gdf(agg={"ShipType": "mode", "MMSI": "mode", "SOG": ["min", "q95", "max"]})
traj_gdf

,traj_id,start_t,end_t,geometry,length,direction,ShipType_mode,MMSI_mode,SOG_min,SOG_q95,SOG_max
0,210035000_0,2017-07-05 17:32:18,2017-07-05 19:28:11,"LINESTRING (11.80462 57.67612, 11.80528 57.676...",3203.316147,58.475166,Cargo,210035000,0.1,9.500,9.8
1,211483120_0,2017-07-05 10:25:53,2017-07-05 10:35:50,"LINESTRING (11.81292 57.64828, 11.81179 57.649...",1525.351240,342.073755,Sailing,211483120,3.4,6.525,6.6
2,211748480_0,2017-07-05 16:11:17,2017-07-05 16:42:18,"LINESTRING (11.82271 57.65239, 11.82555 57.655...",5622.250055,40.585195,Sailing,211748480,4.1,6.835,6.9
3,212048000_0,2017-07-05 02:52:29,2017-07-05 03:21:39,"LINESTRING (11.80488 57.67593, 11.80536 57.676...",3524.918826,50.952378,Tanker,212048000,0.1,8.700,9.1
4,212636000_0,2017-07-05 03:41:59,2017-07-05 06:49:20,"LINESTRING (11.80483 57.67633, 11.80533 57.676...",3496.757824,64.609334,Cargo,212636000,0.1,6.230,10.1
...,...,...,...,...,...,...,...,...,...,...,...
213,266442000_2,2017-07-05 20:07:42,2017-07-05 20:32:22,"LINESTRING (11.80572 57.67610, 11.80659 57.676...",6731.086804,71.215703,Passenger,266442000,5.8,12.000,12.3
214,266442000_3,2017-07-05 21:52:32,2017-07-05 22:16:53,"LINESTRING (11.91209 57.69642, 11.91188 57.696...",6864.690064,251.876542,Passenger,266442000,4.8,11.000,11.2
215,305288000_0,2017-07-05 04:11:46,2017-07-05 04:35:06,"LINESTRING (11.90665 57.69365, 11.90560 57.693...",6309.034737,253.787175,Cargo,305288000,8.0,9.430,9.7
216,376474000_0,2017-07-05 00:03:35,2017-07-05 00:29:14,"LINESTRING (11.91198 57.69591, 11.91146 57.695...",6799.580330,253.065989,Cargo,376474000,7.2,9.300,9.3
